# Libraries

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd 
from functools import partial
from collections import Counter

from joblib import load

# modeling
from sklearn import preprocessing, metrics
from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold
from sklearn.linear_model import LogisticRegressionCV, LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.metrics import recall_score, confusion_matrix
from imblearn.over_sampling import SMOTE

# ignore warnings
import warnings
warnings.filterwarnings(action="ignore")

pd.set_option('display.max_columns', 1000)
pd.set_option('display.max_rows', 1000)

# Get dataset

In [2]:
# get pickle file
providers = load('./data/Providers_Final.pkl')
providers.set_index('Provider', inplace=True)

In [3]:
providers

,PotentialFraud,Perc_Outpatient,DualPatientProvider,Perc_DualPatientType,Ratio_ClaimsPerPatient,Ratio_ClaimsPerAttPhys,Perc_ClaimsPerTopFraudState,Perc_HasTop5AdmtCode,PatientsPerAttPhys,PatientsPerOperPhys,PatientsPerOthPhys,Perc_MultHospAttPhys,Perc_MultHospOperPhys,Perc_MultHospOtherPhys,IP_Perc_Duplicates,IP_Count_UniquePatients,IP_Count_UniqueState,IP_Mean_AgeAtClaim,IP_Perc_HasDied,IP_Perc_GenderZero,IP_Perc_RaceOne,IP_Perc_RaceTwo,IP_Perc_RaceThree,IP_Mean_NumChronicConds,IP_Perc_Alzheimers_Chronic,IP_Perc_Cancer_Chronic,IP_Perc_Depression_Chronic,IP_Perc_Diabetes_Chronic,IP_Perc_HeartFailure_Chronic,IP_Perc_IschemicHeart_Chronic,IP_Perc_KidneyDisease_Chronic,IP_Perc_ObstrPulmonary_Chronic,IP_Perc_Osteoporosis_Chronic,IP_Perc_RheumatoidArthritis_Chronic,IP_Perc_Stroke_Chronic,IP_Perc_HasRenalDisease,IP_Mean_ClaimDuration,IP_Mean_AdmitDuration,IP_Mean_NoOfMonths_PartACov,IP_Mean_NoOfMonths_PartBCov,IP_Mean_ClaimCost,IP_Mean_DailyClaimCost,IP_Mean_DeductibleAmtPaid,IP_Mean_InscClaimAmtReimbursed,IP_Mean_InsReimbursementRatio,IP_Mean_AnnualDeductibleAmt,IP_Mean_AnnualReimbursementAmt,IP_Perc_No_ProcCode,IP_Perc_HasAllPhys,IP_Perc_HasNoPhys,IP_Perc_MultHosp,OP_Perc_Duplicates,OP_Count_UniquePatients,OP_Count_UniqueState,OP_Mean_AgeAtClaim,OP_Perc_HasDied,OP_Perc_GenderZero,OP_Perc_RaceOne,OP_Perc_RaceTwo,OP_Perc_RaceThree,OP_Mean_NumChronicConds,OP_Perc_Alzheimers_Chronic,OP_Perc_Cancer_Chronic,OP_Perc_Depression_Chronic,OP_Perc_Diabetes_Chronic,OP_Perc_HeartFailure_Chronic,OP_Perc_IschemicHeart_Chronic,OP_Perc_KidneyDisease_Chronic,OP_Perc_ObstrPulmonary_Chronic,OP_Perc_Osteoporosis_Chronic,OP_Perc_RheumatoidArthritis_Chronic,OP_Perc_Stroke_Chronic,OP_Perc_HasRenalDisease,OP_Mean_ClaimDuration,OP_Mean_NoOfMonths_PartACov,OP_Mean_NoOfMonths_PartBCov,OP_Mean_ClaimCost,OP_Mean_DailyClaimCost,OP_Mean_DeductibleAmtPaid,OP_Mean_InscClaimAmtReimbursed,OP_Mean_InsReimbursementRatio,OP_Mean_AnnualDeductibleAmt,OP_Mean_AnnualReimbursementAmt,OP_Perc_No_DiagCode,OP_Perc_HasAllPhys,OP_Perc_HasNoPhys,OP_Perc_MultHosp
Provider,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
PRV51001,0,0.800000,1.0,0.400000,1.041667,1.785714,0.0,0.040000,10,19,16,0.240000,0.040000,0.000000,0.000000,5.0,1.0,77.600000,0.000000,0.400000,1.000000,0.000000,0.0,6.000000,0.400000,0.200000,0.800000,0.800000,0.800000,0.800000,0.800000,0.400000,0.000000,0.600000,0.400000,0.400000,6.000000,6.000000,12.000000,12.000000,20468.000000,4077.360000,1068.0,19400.000000,0.878279,897.120000,17606.000000,0.600000,0.2,0.0,0.160000,0.550000,19.0,1.0,77.950000,0.000000,0.350000,0.800000,0.200000,0.000000,5.450000,0.650000,0.200000,0.250000,0.850000,0.750000,0.950000,0.650000,0.400000,0.300000,0.250000,0.200000,0.300000,1.550000,12.000000,12.000000,382.000000,307.000000,0.000000,382.000000,1.000000,463.920000,2615.200000,0.000000,0.050000,0.000000,0.920000
PRV51003,1,0.530303,1.0,0.500000,1.128205,3.000000,0.0,0.060606,73,110,95,0.007576,0.000000,0.000000,0.016129,53.0,3.0,69.935484,0.016129,0.338710,0.790323,0.209677,0.0,4.919355,0.516129,0.112903,0.403226,0.790323,0.580645,0.887097,0.629032,0.370968,0.209677,0.306452,0.112903,0.274194,6.161290,6.161290,11.806452,11.806452,10309.935484,2384.941628,1068.0,9241.935484,0.821059,931.424242,7568.181818,0.370968,0.0,0.0,0.045455,0.357143,66.0,3.0,68.371429,0.000000,0.471429,0.828571,0.157143,0.000000,4.214286,0.342857,0.042857,0.414286,0.728571,0.628571,0.814286,0.357143,0.257143,0.285714,0.271429,0.071429,0.171429,3.357143,11.828571,11.928571,467.714286,336.440760,1.000000,466.714286,0.994032,737.121212,2678.181818,0.000000,0.057143,0.000000,0.818182
PRV51004,0,1.000000,0.0,0.208054,1.079710,3.921053,0.0,0.013423,100,119,112,0.167785,0.013423,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0000

# Upsampling

In [4]:
scores = providers.drop(['PotentialFraud'], axis=1)
scores_cols = scores.columns
scores = preprocessing.normalize(scores, norm='l1')
scores_norm = pd.DataFrame(scores, columns = scores_cols)

decision = providers["PotentialFraud"]

In [5]:
# # Indicies of each class' observations
# i_class0 = np.where(decision == 0)[0] 
# i_class1 = np.where(decision == 1)[0]

# # Number of observations in each class
# n_class0 = len(i_class0)
# n_class1 = len(i_class1)

# n_class0, n_class1


# # For every observation, randomly sample with replacement
# i_class0_upsampled = np.random.choice(i_class1, size=n_class0, replace=True) 
# # Join together class 0's upsampled target vector with class 1's target vector
# decision = np.concatenate((decision[i_class0_upsampled], decision[i_class0]))



# upsample minority class with SMOTE
oversample = SMOTE(random_state = 0)
scores, decision = oversample.fit_resample(scores, decision)

# Count of target variable after upsampling
Counter(decision)

Counter({0: 4904, 1: 4904})

In [6]:
Xtrain, Xtest, ytrain, ytest = train_test_split(scores, decision, random_state=0, stratify = decision)

# Modeling


In [7]:
# some functions that will be used in modeling
def plot_feature_importances(model):
    n_features = scores.shape[1]
    plt.barh(range(n_features), model.feature_importances_, align='center') 
    plt.yticks(np.arange(n_features), scores_cols.to_numpy()) 
    plt.xlabel("Feature importance")
    plt.ylabel("Feature")
    

# Stratified Cross Validation needs to be applied
skf = StratifiedKFold(n_splits = 3, random_state = 0, shuffle = True)

### train a simple classifier model and handling imbalanced classes

In [8]:
logistic_regression = LogisticRegression(random_state=0)
logreg = logistic_regression.fit(Xtrain, ytrain)

print("Training set score: {:.3f}".format(recall_score(ytrain, logreg.predict(Xtrain))))
print("Test set score: {:.3f}".format(recall_score(ytest, logreg.predict(Xtest))))

Training set score: 0.918
Test set score: 0.914


In [9]:
logreg100 = LogisticRegression(C=11, random_state=0).fit(Xtrain, ytrain)
print("Training set score: {:.3f}".format(recall_score(ytrain, logreg100.predict(Xtrain))))
print("Test set score: {:.3f}".format(recall_score(ytest, logreg100.predict(Xtest))))

Training set score: 0.922
Test set score: 0.920


In [10]:
confusion_matrix(ytest,logreg100.predict(Xtest))

array([[ 864,  362],
       [  98, 1128]])

In [11]:
param_grid = [{'C': np.logspace(-1,1,100)}]
logreg100cv = GridSearchCV(logreg100, param_grid = param_grid).fit(Xtrain,ytrain)

### decision tree

In [ ]:
tree = DecisionTreeClassifier(max_depth=4, random_state=0) 
tree.fit(Xtrain, ytrain)
print("Accuracy on training set: {:.3f}".format(tree.score(Xtrain, ytrain))) 
print("Accuracy on test set: {:.3f}".format(tree.score(Xtest, ytest)))

display(recall_score(ytrain, tree.predict(Xtrain)))
display(recall_score(ytest, tree.predict(Xtest)))

In [ ]:
plt.figure(figsize=(10,20))
plot_feature_importances(tree)

### random forest

In [12]:
forest = RandomForestClassifier(n_estimators=100, random_state=0, max_depth = 30, max_features = 5) 
forest.fit(Xtrain, ytrain)

display(recall_score(ytrain, forest.predict(Xtrain)))
display(recall_score(ytest, forest.predict(Xtest)))

Accuracy on training set: 1.000
Accuracy on test set: 0.960


1.0

0.9722675367047309

In [13]:
param_grid = {
    'n_estimators': [200, 600, 1000, 1400, 2000],
    'max_features': ['auto', 'sqrt'],
    'min_samples_split': [2, 4, 6]
}

grid_search = GridSearchCV(RandomForestClassifier(random_state=0), param_grid, cv=skf)
grid_search.fit(Xtrain, ytrain)

print("Best parameters: {}".format(grid_search.best_params_))
print("Best cross-validation score: {:.2f}".format(grid_search.best_score_))
print("Best estimator:\n{}".format(grid_search.best_estimator_))

Best parameters: {'max_features': 'auto', 'min_samples_split': 2, 'n_estimators': 600}
Best cross-validation score: 0.95
Best estimator:
RandomForestClassifier(n_estimators=600, random_state=0)


In [14]:
display(recall_score(ytrain, grid_search.best_estimator_.predict(Xtrain)))
display(recall_score(ytest, grid_search.best_estimator_.predict(Xtest)))

1.0

0.9771615008156607

In [ ]:
plt.figure(figsize=(10,20))
plot_feature_importances(grid_search.best_estimator_)

### gradient boosting

In [ ]:
gbrt = GradientBoostingClassifier(random_state=0)
gbrt.fit(Xtrain, ytrain)

print("Accuracy on training set: {:.3f}".format(gbrt.score(Xtrain, ytrain)))
print("Accuracy on test set: {:.3f}".format(gbrt.score(Xtest, ytest)))

In [ ]:
gbrt = GradientBoostingClassifier(random_state=0, max_depth=1) 
gbrt.fit(Xtrain, ytrain)

print("Accuracy on training set: {:.3f}".format(gbrt.score(Xtrain, ytrain))) 
print("Accuracy on test set: {:.3f}".format(gbrt.score(Xtest, ytest)))

In [ ]:
gbrt = GradientBoostingClassifier(random_state=0, learning_rate=0.01) 
gbrt.fit(Xtrain, ytrain)

print("Accuracy on training set: {:.3f}".format(gbrt.score(Xtrain, ytrain))) 
print("Accuracy on test set: {:.3f}".format(gbrt.score(Xtest, ytest)))

In [ ]:
gb_param_grid = {
    'learning_rate':[0.1,0.05,0.01], 
    'n_estimators':[100,500,1000,1500],
    'max_depth':[2,4,6]
}

gb_grid_search = GridSearchCV(GradientBoostingClassifier(random_state=0), gb_param_grid, cv=3)
gb_grid_search.fit(Xtrain, ytrain)

print("Best parameters: {}".format(gb_grid_search.best_params_))
print("Best cross-validation score: {:.2f}".format(gb_grid_search.best_score_))
print("Best estimator:\n{}".format(gb_grid_search.best_estimator_))

In [ ]:
display(recall_score(ytrain, gb_grid_search.best_estimator_.predict(Xtrain)))
display(recall_score(ytest, gb_grid_search.best_estimator_.predict(Xtest)))

In [ ]:
plt.figure(figsize=(10,20))
plot_feature_importances(gb_grid_search.best_estimator_)

### support vector machine

In [ ]:
svc = SVC() 
svc.fit(Xtrain, ytrain)

print("Accuracy on training set: {:.2f}".format(svc.score(Xtrain, ytrain))) 
print("Accuracy on test set: {:.2f}".format(svc.score(Xtest, ytest)))

In [ ]:
svc = SVC(C=1000) 
svc.fit(Xtrain, ytrain)

print("Accuracy on training set: {:.2f}".format(svc.score(Xtrain, ytrain))) 
print("Accuracy on test set: {:.2f}".format(svc.score(Xtest, ytest)))